# Starting with just 2023 for my laptop 

In [2]:
import duckdb
import pandas as pd
import pandas_ta as ta

con = duckdb.connect(r"C:\Users\rybot\OneDrive\Databases\datadump.duckdb")
print(con)

In [3]:
tables = con.execute('PRAGMA show_tables;').fetchall()
print(tables)

[('Stocks',)]


In [4]:
qry = '''
    SELECT * FROM Stocks
    WHERE EXTRACT(YEAR FROM Datetime) = 2023 
    AND Stock = 'AAPL' 
    AND Interval = 1
'''

df = con.execute(qry).fetchdf()
# Sort the Data from oldest to newest
df = df.sort_values(by='Datetime').reset_index(drop=True)


print(df.head())

             Datetime        open        high         low       close  volume  \
0 2023-01-03 04:00:00  129.391006  130.113998  129.386002  130.110001  5381.0   
1 2023-01-03 04:01:00  129.977005  130.283005  129.972000  130.209000  6072.0   
2 2023-01-03 04:02:00  130.285004  130.352005  130.270004  130.279007  1053.0   
3 2023-01-03 04:03:00  130.294998  130.401993  130.289993  130.388000  1754.0   
4 2023-01-03 04:04:00  130.384003  130.570999  130.378998  130.567001  3766.0   

  Stock  interval  
0  AAPL       1.0  
1  AAPL       1.0  
2  AAPL       1.0  
3  AAPL       1.0  
4  AAPL       1.0  


# Creating and Flattening the Data for:
#### Packet Size (Only 1 min intervals)

### Getting the Data, and calculating P/L

In [5]:
df['Time'] = df['Datetime'].dt.time
start_time = pd.to_datetime('09:00:00').time()
end_time = pd.to_datetime('16:00:00').time()
TradingDay_df = df[(df['Time']>start_time)&(df['Time']<end_time)]
print(TradingDay_df.info())

def trail_stoploss(index, tradingdata, whole_percentage):
    idx = index
    td = tradingdata
    pct = whole_percentage/100
    max_holdtime = 30 # Only hold the trade for twenty minutes

    after_td = td.loc[idx:idx+max_holdtime+1] # Get the dataframe from the row you want until 100 minutes later
    
    basis = td['close'].loc[idx]
    max_price = basis   # Initially the max price is the basis - No Shorting to Start
    min_price = basis - (basis*pct)
    time_stop = idx + max_holdtime 
    time_counter = idx
    
    #print(f"Basis: {basis}")

    for i, row in after_td.iterrows():
        close = row['close']
        #print(f"Close: {close}, Max: {max_price}, Min: {min_price}, Counter: {time_counter-idx}")
        if close > max_price:
            max_price = close
            min_price = max_price - (max_price * pct)
        elif close < min_price:
            profit = (close - basis) / basis
            return [i, profit]
        
        time_counter += 1
        if time_counter > time_stop:
            break

    # If the loop ends without triggering stop loss, calculate the profit based on the last close
    profit = (close - basis) / basis
    return [i, profit]
            
    




               Datetime        open        high         low       close  \
235 2023-01-03 09:01:00  130.037003  130.113998  129.942001  129.960999   
236 2023-01-03 09:02:00  129.936996  129.964996  129.882996  129.921005   
237 2023-01-03 09:03:00  129.917007  129.964996  129.882996  129.910995   
238 2023-01-03 09:04:00  129.897995  129.934998  129.873001  129.882004   
239 2023-01-03 09:05:00  129.927002  129.964996  129.875000  129.884003   

      volume Stock  interval      Time  
235  14208.0  AAPL       1.0  09:01:00  
236   3281.0  AAPL       1.0  09:02:00  
237   3530.0  AAPL       1.0  09:03:00  
238   5429.0  AAPL       1.0  09:04:00  
239   1600.0  AAPL       1.0  09:05:00  


In [34]:
print(TradingDay_df.index[0])

235


In [18]:
import os
import csv

columns = []
flattened_df100 = []
columns100 = []
flattened_df50 = []
columns50 = []
flattened_df20 = []
columns20 = []
flattened_df10 = []
columns10 = []

First_Index = TradingDay_df.index[0]
Index_Counter = First_Index

# Creating the CSV Files
# 100
packet = df[['open', 'close']].loc[First_Index-99:First_Index]
for i in range(len(packet)):
        columns100.append(f'open_{i+1}')
        columns100.append(f'close_{i+1}')
columns100.extend(["EMA100","EMA20","RSI","Profit"])
with open('packet100.csv', 'a', newline='') as f:
      csv.writer(f).writerow(columns100)

# 50
packet = df[['open', 'close']].loc[First_Index-49:First_Index]
for i in range(len(packet)):
        columns50.append(f'open_{i+1}')
        columns50.append(f'close_{i+1}')
columns50.extend(["EMA50","EMA20","RSI","Profit"])
with open('packet50.csv', 'a', newline='') as f:
      csv.writer(f).writerow(columns50)

# 20
packet = df[['open', 'close']].loc[First_Index-19:First_Index]
for i in range(len(packet)):
        columns20.append(f'open_{i+1}')
        columns20.append(f'close_{i+1}')
columns20.extend(["EMA20","EMA5","RSI","Profit"])
with open('packet20.csv', 'a', newline='') as f:
      csv.writer(f).writerow(columns20)

#10
packet = df[['open', 'close']].loc[First_Index-9:First_Index]
for i in range(len(packet)):
        columns10.append(f'open_{i+1}')
        columns10.append(f'close_{i+1}')
columns10.extend(["EMA10","RSI","Profit"])
with open('packet10.csv', 'a', newline='') as f:
      csv.writer(f).writerow(columns10)



for index, row in TradingDay_df.iterrows():
    
    if index % 5000 ==0:
         print(index)

    profit = trail_stoploss(index, df, 5)[0]

    # 100 Minutes
    packet100 = df[['open', 'close']].loc[index-99:index]
    flatpack100 = list(packet100.values.flatten())

    packet100['EMA100'] = ta.ema(packet100['close'], length=100)
    packet100['EMA20'] = ta.ema(packet100['close'], length=20)
    packet100['RSI'] = ta.rsi(packet100['close'])

    TA_100 = list(packet100[['EMA100','EMA20','RSI']].loc[index])
    flatpack100.extend(TA_100)
    flatpack100.append(profit)
    with open('packet100.csv', 'a', newline='') as f:
      csv.writer(f).writerow(flatpack100)

    

    # 50 Minutes
    packet50 = df[['open', 'close']].loc[index-49:index]
    flatpack50 = list(packet50.values.flatten())

    # Trend Analysis
    packet50['EMA50'] = ta.ema(packet50['close'], length=50)
    packet50['EMA20'] = ta.ema(packet50['close'], length=20)
    packet50['RSI'] = ta.rsi(packet50['close'])

    TA_50 = list(packet50[['EMA50','EMA20','RSI']].loc[index])
    flatpack50.extend(TA_50)
    flatpack50.append(profit)
    with open('packet50.csv', 'a', newline='') as f:
      csv.writer(f).writerow(flatpack50)


    # 20 Minutes
    packet20 = df[['open', 'close']].loc[index-19:index]
    flatpack20 = list(packet20.values.flatten())

    # Trend Analysis
    packet20['EMA20'] = ta.ema(packet20['close'], length=20)
    packet20['EMA5'] = ta.ema(packet20['close'], length=5)
    packet20['RSI'] = ta.rsi(packet20['close'])

    TA_20 = list(packet20[['EMA20','EMA5','RSI']].loc[index])
    flatpack20.extend(TA_20)
    flatpack20.append(profit)
    with open('packet20.csv', 'a', newline='') as f:
      csv.writer(f).writerow(flatpack20)


    # 10 Minutes   

    packet10 = df[['open', 'close']].loc[index-9:index]
    flatpack10 = list(packet10.values.flatten())

    # Trend Analysis
    packet10['EMA10'] = ta.ema(packet10['close'], length=10)
    packet10['RSI'] = ta.rsi(packet10['close'], length=8)

    TA_10 = list(packet10[['EMA10','RSI']].loc[index])
    flatpack10.extend(TA_10)
    flatpack10.append(profit)
    with open('packet10.csv', 'a', newline='') as f:
      csv.writer(f).writerow(flatpack10)
    

    


10000
15000
20000
30000
40000
45000
50000
55000
60000
70000
75000
80000
100000
115000
120000
140000
145000
160000
165000
180000
185000
200000
220000


In [ ]:
index = 235

TA_20 = list(packet100[['EMA20','EMA5','RSI']].loc[index])